# Max cut problem

## cuTensorNet

https://github.com/NVIDIA/cuQuantum/blob/main/python/samples/cutensornet/circuit_converter/qiskit_advanced.ipynb

In [ ]:
from __future__ import annotations

import time
from collections.abc import Sequence
import matplotlib.pyplot as plt

import numpy as np
from scipy.optimize import minimize
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import QAOAAnsatz
from qiskit_aer import AerSimulator

import cupy as cp

from qmlant.neural_networks import EstimatorTN
from qmlant.models.vqe import (
    SimpleQAOA, IsingConverter, HamiltonianConverter, QAOAMixer, circuit_to_einsum_expectation
)
from qmlant.visualization import draw_quimb_tn

## QAOA Quantum Circuit

In [ ]:
n_qubits = 5
n_reps = 3

betas = ParameterVector("β", n_reps)
gammas = ParameterVector("γ", n_reps)
beta_idx = iter(range(n_reps))
bi = lambda: next(beta_idx)
gamma_idx = iter(range(n_reps))
gi = lambda: next(gamma_idx)

qc = QuantumCircuit(n_qubits)
qc.h(qc.qregs[0][:])
for _ in range(n_reps):
    gamma = gammas[gi()]
    qc.rzz(gamma, 0, 1)
    qc.rzz(gamma, 0, 2)
    qc.rzz(gamma, 1, 3)
    qc.rzz(gamma, 2, 3)
    qc.rzz(gamma, 2, 4)
    qc.rzz(gamma, 3, 4)
    qc.barrier()
    beta = betas[bi()]
    for i in range(n_qubits):
        qc.rx(beta, i)

qc.draw(fold=-1)

In [ ]:
hamiltonian = ["ZZIII", "ZIZII", "IZIZI", "IIZZI", "IIZIZ", "IIIZZ"]

expr, operands, pname2locs = circuit_to_einsum_expectation(
    qc, hamiltonian, qaoa_mixer=QAOAMixer.X_MIXER
)
make_pname2theta = operands["make_pname2theta"]

## Optimization

In [ ]:
losses = []
count = 0
estimator = EstimatorTN(pname2locs, expr, operands)

def compute_expectation_tn(params, *args):
    global count, n_reps
    (estimator,) = args
    time_start = time.time()
    energy = estimator.forward(params)
    if count % 50 == 0:
        print(f"[{count}] {energy} (elapsed={round(time.time() - time_start, 3)}s)")
    count += 1

    losses.append(energy)

    return energy

In [ ]:
%%time

rng = np.random.default_rng(42)
init = rng.random(qc.num_parameters) * 2*np.pi

result = minimize(
    compute_expectation_tn,
    init,
    args=(estimator,),
    method="COBYLA",
    options={
        "maxiter": 500
    },
)

print(result.message)
print(f"opt value={round(result.fun, 3)}")

## Validate Results

In [ ]:
mapping = make_pname2theta(result.x)
parameter2value = {param: mapping[param.name] for param in qc.parameters}
opt_qc = qc.bind_parameters(parameter2value)
opt_qc.measure_all()

sim = AerSimulator()
t_qc = transpile(opt_qc, backend=sim)
counts = sim.run(t_qc).result().get_counts()
for k, n in sorted(counts.items(), key=lambda k_v: -k_v[1]):
    if n < 100:
        continue
    print(k[::-1], n)

## Visualization

In [ ]:
plt.figure()
x = np.arange(0, len(losses), 1)
plt.plot(x, losses, color="blue")
plt.show()

In [ ]:
qubit_op = SparsePauliOp([ham[::-1] for ham in hamiltonian])
draw_quimb_tn(qc, qubit_op, True)